In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip uninstall --yes diffusers
%pip install git+https://github.com/neverix/diffusers@patch-4

Found existing installation: diffusers 0.25.0.dev0
Uninstalling diffusers-0.25.0.dev0:
  Successfully uninstalled diffusers-0.25.0.dev0
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/neverix/diffusers (to revision patch-4) to /tmp/pip-req-build-ncqqvq6c
  Running command git clone --filter=blob:none --quiet https://github.com/neverix/diffusers /tmp/pip-req-build-ncqqvq6c
  Running command git checkout -b patch-4 --track origin/patch-4
  Switched to a new branch 'patch-4'
  Branch 'patch-4' set up to track remote branch 'patch-4' from 'origin'.
  Resolved https://github.com/neverix/diffusers to commit 943d95c2eb53dfc20ac1185d73d61fe976119725
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.25.0.dev0-py3-none-any.whl size=1767474 sha256=f0973a3b790535df7ae701f95eb81497c8b3d3ee612834093f34b8ca57c16185
  St

In [1]:
!rm -rf ~/.cache/huggingface/hub
# !rm -rf cache

In [3]:
# !mkdir -p cache
# !cd cache && GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0 --depth=1
# !cd cache/stable-diffusion-xl-base-1.0 && git lfs pull --include '*.msgpack' && git lfs pull --include '*.model' && git lfs pull --include '*.json'

fatal: destination path 'stable-diffusion-xl-base-1.0' already exists and is not an empty directory.


In [1]:
from diffusers import FlaxStableDiffusionXLPipeline
import jax.numpy as jnp


dtype = jnp.bfloat16
pipe, params = FlaxStableDiffusionXLPipeline.from_pretrained(
    # "nev/lcm-sdxl-flax",
    # "nev/lcm-sdxl-pt",
    "nev/sdxl-turbo-pt",
    # "stabilityai/stable-diffusion-xl-base-1.0",
    # "cache/stable-diffusion-xl-base-1.0",
    # "pcuenq/stable-diffusion-xl-base-1.0-flax",
    # "stabilityai/sdxl-turbo",
    from_pt=True,
    dtype=jnp.float32
)

/home/neverix/.pyenv/versions/3.12.0/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 78073.91it/s]


In [2]:

import jax
print(jax.devices())

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]


In [3]:
params.keys()

dict_keys(['text_encoder', 'text_encoder_2', 'unet', 'vae', 'scheduler'])

In [4]:
scheduler_state = params.pop("scheduler")
vae_state = params.pop("vae")
params = jax.tree_util.tree_map(lambda x: x.astype(dtype), params)
params["scheduler"] = scheduler_state
params["vae"] = vae_state

In [5]:
from flax.training.common_utils import shard
from flax.jax_utils import replicate
import numpy as np


p_params = replicate(params)

def create_key(seed=0):
    return jax.random.PRNGKey(seed)

rng = create_key(0)
rng = jax.random.split(rng, jax.device_count())

do_jit = True

# def generate(prompt_ids, neg_prompt_ids):
#     images = pipe(
#         prompt_ids if do_jit else prompt_ids[0],
#         p_params if do_jit else params,
#         rng if do_jit else rng[0],
#         num_inference_steps=1,
#         # neg_prompt_ids=neg_prompt_ids if do_jit else neg_prompt_ids[0],
#         guidance_scale = 1.0,
#         jit=do_jit,
#     ).images
#     images = images.reshape((images.shape[0] * images.shape[1], ) + images.shape[-3:])
#     return pipe.numpy_to_pil(np.array(images))

In [6]:
imgs_per_device = 1

prompt = "a cat"
prompts = [prompt] * jax.device_count() * imgs_per_device
prompt_ids = pipe.prepare_inputs(prompts)
prompt_ids = shard(prompt_ids)

# neg_prompt = "misshapen"
# neg_prompts = [neg_prompt] * jax.device_count() * imgs_per_device
# neg_prompt_ids = pipe.prepare_inputs(neg_prompts)
# neg_prompt_ids = shard(neg_prompt_ids)

# for u in generate(prompt_ids, neg_prompt_ids):
#     display(u)

In [7]:
prompt_embeds, pooled_embeds = jax.vmap(pipe.get_embeddings, in_axes=(0, None))(prompt_ids, params)

In [8]:
height, width = 512, 512
add_time_ids = pipe._get_add_time_ids(
    (height, width), (0, 0), (height, width), prompt_embeds.shape[0], dtype=prompt_embeds.dtype
)

In [9]:
batch_size = prompt_embeds.shape[0]
latents_shape = (
    batch_size,
    pipe.unet.config.in_channels,
    height // pipe.vae_scale_factor,
    width // pipe.vae_scale_factor,
)
latents = jax.vmap(lambda x: jax.random.normal(x, shape=latents_shape, dtype=jnp.float32))(rng)

In [10]:
added_cond_kwargs = {"text_embeds": pooled_embeds, "time_ids": add_time_ids}
noise_pred = jax.pmap(pipe.unet.apply, in_axes=(0, 0, None, 0, 0))(
    {"params": p_params["unet"]},
    jnp.array(latents),
    jnp.array(pipe.scheduler.config.num_train_timesteps - 1, dtype=jnp.int32),
    prompt_embeds,
    added_cond_kwargs,
).sample

XlaRuntimeError: RESOURCE_EXHAUSTED: Error allocating device buffer: Attempting to allocate 2.50G. That was not possible. There are 661.25M free.; (0x0x0_HBM0)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(noise_pred[0, 0])
plt.show()

NameError: name 'noise_pred' is not defined

In [ ]:
#  * scheduler_state.init_noise_sigma

In [21]:
from inspect import getsource
print(getsource(pipe.__call__))

    def __call__(
        self,
        prompt_ids: jax.Array,
        params: Union[Dict, FrozenDict],
        prng_seed: jax.Array,
        num_inference_steps: int = 50,
        guidance_scale: Union[float, jax.Array] = 7.5,
        height: Optional[int] = None,
        width: Optional[int] = None,
        latents: jnp.array = None,
        neg_prompt_ids: jnp.array = None,
        return_dict: bool = True,
        output_type: str = None,
        jit: bool = False,
    ):
        # 0. Default height and width to unet
        height = height or self.unet.config.sample_size * self.vae_scale_factor
        width = width or self.unet.config.sample_size * self.vae_scale_factor

        if isinstance(guidance_scale, float) and jit:
            # Convert to a tensor so each device gets a copy.
            guidance_scale = jnp.array([guidance_scale] * prompt_ids.shape[0])
            guidance_scale = guidance_scale[:, None]

        return_latents = output_type == "latent"

        if jit: